In [22]:
import requests
import torch
import torch.nn as nn
from torch.nn import functional as F

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
with open("input.txt", "wb") as f:
   f.write(response.content)

In [23]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [24]:
print(text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [25]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"There are {vocab_size} unique characters. They're as below:")
print(''.join(chars))

There are 65 unique characters. They're as below:

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [26]:
"""
Create mappings, string to integer (stoi) and integer to string (itos)
"""
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
# encode : string -> list of integers
encode = lambda s: [ stoi[c] for c in s ]
# decode : list of integers -> string
decode = lambda l: ''.join([ itos[i] for i in l ])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [27]:
# Encode the entire text and store in a torch.tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:148])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0])


In [28]:
n = int(0.9*len(data))

train_data = data[:n]
val_data = data[n:]

In [29]:
# size of the training block
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [30]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [31]:
torch.manual_seed(1337)
batch_size = 4 # number of independent sequences to be processed parallely
block_size = 8 # maximum context length

def get_batch(split):
    data = train_data if split == 'train' else val_data

    # generate a tensor of values between 0 and len(data)-block_size with batch_size columns
    # this will give us a random set of data to train on from the training data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    # get the blocks for each starting index in ix
    # get the next elements to each token in x
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("\ntargets:")
print(yb.shape)
print(yb)
print("----")

for b in range(batch_size):
    for t in range(block_size):
        # from xb tensor, get the values up to t+1 at row b
        context = xb[b, :t+1]
        # from yb tensor, get the t column of row b
        target = yb[b,t]
        print(f"when input is {context.tolist()}, the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24], the target is 43
when input is [24, 43], the target is 58
when input is [24, 43, 58], the target is 5
when input is [24, 43, 58, 5], the target is 57
when input is [24, 43, 58, 5, 57], the target is 1
when input is [24, 43, 58, 5, 57, 1], the target is 46
when input is [24, 43, 58, 5, 57, 1, 46], the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43], the target is 39
when input is [44], the target is 53
when input is [44, 53], the target is 56
when input is [44, 53, 56], the target is 1
when input is [44, 53, 56, 1], the target is 58
when input is [44, 53, 56, 1, 58]

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)